In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 46.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.1-py3-none-any.whl size=53220 sha256=103a4c1c8f8e4e4159f5ff19e15b991a2af4603c576f54e75bc1082c74388898
  Stored in directory: /root/.cache/pip/wheels/2f/23/d1/e9c8f5f2e8a61bb4cb8e4fdd0d0c951dab78882e3807c7f7ca
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646666 sha256=48b923da971b829f9a420951d6e864ff46e42d8bcaeec170c211dfb03a1424ca
  Stored in directory: /root/.cache/pip/wheels/5c/91/bf/14eed6eafd0a83f76eab

In [ ]:
import numpy as np
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer

----
# Data
---

In [ ]:
path = '/content/drive/MyDrive/2.Study/BERT/KCC150_K01.txt'

data = []
with open(path, 'r', encoding='cp949') as f:
  for _ in range(4000):
    data.append(f.readline().strip())

data

---
# Preprocessing
---

In [ ]:
def extract_word(text):
  text = text.lower() # 소문자 변환
  convert = re.compile('^\d*\d$|[^가-힣a-zA-Z0-9.ㅋㅜㅎㅠ]') # 해당 문자만 남기고 나머지 제거
  result = convert.sub(' ',text) # 변환

  result = re.sub(r'[" "]+', " ",result) # 공백 여러개를 한개로 변환
  result = result.strip() # 양쪽 공백 제거

  return result

In [ ]:
for i in range(len(data)):
  data[i] = extract_word(data[i])

In [ ]:
with open('/content/drive/MyDrive/2.Study/BERT/stopwords.txt', 'r', encoding = 'utf-8') as f:
  stopword_list = []
  value = f.readlines()
  for i in value:
    stopword_list.append(i.strip())

stopword_list[:4]

['가', '가까스로', '가령', '각']

---
# Morphological analysis
---

In [ ]:
!python -m spacy download ko_core_news_lg
import spacy
nlp = spacy.load('ko_core_news_lg')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-23 07:43:31.405154: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 07:43:32.643678: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-23 07:43:32.643812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
def tokenizer(text):
    doc = nlp(text)

    vlaue = []
    # 토큰의 정규화된 형태소
    texts_list = [token.lemma_.split('+') for token in doc]
    # 토큰의 형태소 태그
    tags_list = [token.tag_.split('+') for token in doc]

    for token_idx in range(len(doc)):
        if tags_list[token_idx][0] in ('ncn','ncpa','ncps','nc','nq','f'):
        # 현재 토큰의 정규화된 형태서와 형태소 태그의 개수가 동일하다면 
            if len(texts_list[token_idx]) == len(tags_list[token_idx]):
                vlaue
                if tags_list[token_idx][0].startswith('n') or tags_list[token_idx][0].startswith('f'):
                    vlaue.append(''.join([text for tag,text in zip(tags_list[token_idx], texts_list[token_idx]) if tag in ('ncn','ncpa','ncps','nc','nq','f')]))
                else:
                    vlaue.append(texts_list[token_idx][0] + '다')
            elif tags_list[token_idx][0] in ('ncn','ncpa','ncps','nc','f'):
                vlaue.append(texts_list[token_idx][0])
            else:
                vlaue.append(''.join((texts_list[token_idx])).replace(' ','_'))

    return vlaue

In [ ]:
# BOW 생성
vectorizer = TfidfVectorizer(tokenizer = tokenizer, min_df = 2, ngram_range = (1,1), stop_words = stopword_list)

In [ ]:
vectorizer.fit(data)
vectorizer.vocabulary_

---
# BERTopic
---

In [ ]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp39-cp39-linux_x86_64.whl size=3582115 sha256=867d11a6e4613f12ed78219ccd1e3b27d7fbe6de7908b29d69345a67f20df0bd


In [ ]:
from bertopic import BERTopic
from umap import UMAP
import hdbscan

---
### UMAP
---

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcIJ0Jd%2FbtrxrbQB0YD%2FqF4Am3Y4DFvFTEBvIN6vHK%2Fimg.png'><img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FYNnw2%2FbtrxAa3tHdy%2FbfnSWgdePxJNhgkGi5A1vk%2Fimg.png'>

- PCA와 비슷한 차원 축소 알고리즘

    - PCA의 경우 maxtirx factorization이고 

- UMAP은 neighbor graph를 기반의 비선형 차원축소 기법

    - 고차원 데이터셋을 이해하고 시각화하는 툴 

    - t-SNE와 비슷하나 속도도 더 빠르고, 데이터의 global 구조를 더 잘 보존함
- 즉, 데이터가 실제로 존재하는 n차원의 그래프를 더 낮은 차원의 비슷한 그래프로 축소시키는 것

- 일반적으로 클러스터링 알고리즘은 고차원의 공간에서 클러스트링을 수행하는데 어려움을 겪기 때문에

- UMAP을 통해 임베딩의 local 및 global strcture를 보존하면서 차원을 축소 

In [ ]:
'''n_neighbor = 이웃한 데이터 포인트의 수를 나타내는 파리미터, 일반적으로 작을수록 고차원의 데이터를 잘 표현

   min_dist = 최소 거리를 나타내는 파라미터, 작을수록 저차원 공간에서의 데이터 간 거리가 더 유지

   n_components = 차원 축소 후 저차원 공간의 차원 수를 지정하는 파라미터, 일반적으로 2차원 혹은 3차원으로 시각화 하는데 사용

   metric = 거리 측정 방법을 지정하는 파라미터, 유클리드, 멘하턴, 코사인 등등

   n_epoch = 최적화 과정에서 반복 횟수를 지정, 크면 정확한 변환 결과를 얻지만 계산 비용 증가

   learning_rate = 학습률, 클수록 더 큰 변환을 얻음'''

umap_model = UMAP(n_neighbors= 15, min_dist = 0.1, n_components = 5, random_state = 415, metric = 'euclidean')

---
### hdbscan
---

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcSBFNR%2Fbtrxrbwkcud%2FctVHPNTlow5YD2MtKk8Pi0%2Fimg.jpg'>


- 밀도 기반 공간 군집화 기법 중 하나로 DBSCAN과 유사하지만, 밀도 기반 클러스터링 기법을 더욱 효과적으로 처리하도록 개선된 버전

- 축소된 임베딩을 HDBSCAN을 사용해 outlier를 식별하고 클러스터링

- 고차원의 nested clusters를 효과적으로 분류하기 위한 알고리즘

- 핵심 개념은 Mininum Spanning Tree(MST)로 거리 매트릭스를 기반으로 MST를 구성하고 군집을 구성하구 위해 MST를 잘라내는 방식

- MST에서 각 포인트에 대한 최소연결 거리를 계산하면 클러스터링 결과를 얻을 수 있게 되며, 가우시안 혼합 모델을 사용하여 클러스터를 분리하는 방법을 적용

- HDBSCAN의 이점은 자동 클러스터 크기 결정과 높은 노이즈 저항성

[HDBSCAN 설명](https://heave.tistory.com/60)

In [ ]:
'''min_cluster_size = 클러스터를 형성하는데 필요한 최소 포인트 수 설정, 클수록 더 밀집된 클러스터 생성

   metirc = 데이터 포인트 사이의 거리를 측정하는데 사용할 거리 메트릭

   cluster_selection_method = HDBSCAN에 의해 생성된 계층적 클러스터 트리에서 클러스터의 최종세트를 선택하기 위한 기준을 지정
      2가지 주요 옵션은 eom(질량 초과)와 leaf로 
      eom 방법은 총 데이터 질량의 지정된 백분율을 캡처하는데 필요한 계층 구조의 최소 수준을 기반으로 클러스터를 선택
      leaf 방법은 최소 포인트 수가 있는 계층 구조의 최하위 수준을 기준으로 클러스터를 선택 

   predict_data = 훈련된 모델을 기반으로 기존 클러스터에 새 데이터 포인트를 할당, 전체 모델을 재교육하지 않고 새 데이터 포인트의 클러스터 구성원을 예측하는데 유용

   min_samples = 그래프를 작성할때 고려할 가장 가까운 이웃의 수를 설정, 커질수록 포인트 간의 연결이 더많아지고 더 많은 클러스터가 연결될 수 있지만 시간이 늘어나게 됨

   Alpha = 클러스터 밀도와 클러스터 수 사이의 균형을 제어, 커질수록 클러스터가 점점 더 작아지며 대부분 기본값 1.0이면 충분

   Bandwidth = 밀도 추정에 사용되는 커널의 너비를 제어, 커질수록 밀도 추정치가 더 부드러워지고 연결된 클러스터가 더 많아지게 됨

   Leaf_size = 리프 노드 크기를 제어, 커질수록 클러스터링 시간이 빨라지지만 품질이 낮아지게됨'''
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size = 10, metric = 'euclidean', cluster_selection_method = 'eom', prediction_data = True)

In [ ]:
model = BERTopic(embedding_model = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
                 vectorizer_model = vectorizer,
                 top_n_words = 15,
                 min_topic_size = 10,
                 umap_model = umap_model,
                 hdbscan_model = hdbscan_model,
                 verbose = True)

In [ ]:
topics, probs = model.fit_transform(data)

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

2023-03-23 10:02:13,753 - BERTopic - Transformed documents to Embeddings
2023-03-23 10:02:41,627 - BERTopic - Reduced dimensionality
2023-03-23 10:02:42,071 - BERTopic - Clustered reduced embeddings


In [ ]:
# 토픽의 개수, 토픽의 크기, 각 토픽에 할당된 단어
# -1의 값은 토픽에 포함되지 않는 문서
model.get_topic_info()

,Topic,Count,Name
0,-1,2428,-1_경찰_전_한국_검찰
1,0,283,0_수술_말_장애인_꿈
2,1,275,1_소리_사람_여자_부정
3,2,157,2_올해_증가_상승_지난해
4,3,122,3_대출_은행_주가_돈
5,4,121,4_대구_서울_교수_대전
6,5,120,5_타자_안타_시즌_타석
7,6,80,6_일치_베트남_효과_건강
8,7,51,7_학교_학습_수업_참여
9,8,38,8_숲_세목_유치_공동


In [ ]:
model.get_topic(10)

[('부진', 0.09636612258442384),
 ('초반', 0.08554432761531158),
 ('넥센', 0.0726637147087419),
 ('실수', 0.06711326011105447),
 ('경기', 0.06703052749797436),
 ('최악', 0.061081672322790215),
 ('통증', 0.059341411739098994),
 ('구단', 0.05354663221621141),
 ('패배', 0.052193560515890684),
 ('lg', 0.05139840597766953),
 ('호소', 0.04705965804121242),
 ('내년', 0.0456944337251392),
 ('선발', 0.04510294063787924),
 ('시즌', 0.04343423235221933),
 ('이후', 0.03707351289368887)]

---
# Visualize
---

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_heatmap()